In [ ]:
import h5py
import pandas as pd
# import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import io
import base64

%matplotlib inline

POLLUTANT_MAP = {
    "1": "SO2", "5": "NO2", "7": "NO", "8": "NOx", "9": "CO",
    "10": "Benzene", "38": "PM10", "6001": "PM2.5", "6006": "O3"
}

In [31]:
upload_output = widgets.Output()
station_dropdown = widgets.Dropdown(description="Station:", layout=widgets.Layout(width="50%"))
pollutant_dropdown = widgets.Dropdown(description="Pollutant:", layout=widgets.Layout(width="50%"))
plot_button = widgets.Button(description="📈 Plot", button_style="success")
output_plot = widgets.Output()
file_upload = widgets.FileUpload(accept=".h5", multiple=False, description="Upload .h5 File")


In [ ]:
def update_gui_from_file(h5_path):
    with h5py.File(h5_path, "r") as f:
        station_pollutant_map = {
            group: list(f["FR"][group].keys()) for group in f["FR"]
        }

    station_dropdown.options = list(station_pollutant_map.keys())

    def update_pollutants(*args):
        station = station_dropdown.value
        codes = station_pollutant_map.get(station, [])
        pollutant_dropdown.options = [(POLLUTANT_MAP.get(p, p), p) for p in codes]

    station_dropdown.observe(update_pollutants, names="value")
    update_pollutants()

    def on_plot(b):
        output_plot.clear_output()
        station = station_dropdown.value
        pollutant = pollutant_dropdown.value

        with h5py.File(h5_path, "r") as f:
            grp = f[f"FR/{station}/{pollutant}"]
            timestamps = [ts.decode("utf-8") for ts in grp["timestamps"][:]]
            values = grp["values"][:]
            unit = grp.attrs.get("unit", "")
            agg = grp.attrs.get("agg_type", "")
            val = grp.attrs.get("validity", "")

        df = pd.DataFrame({"timestamp": pd.to_datetime(timestamps), "value": values})
        label = POLLUTANT_MAP.get(pollutant, pollutant)

        with output_plot:
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=df["timestamp"], y=df["value"],
                mode="lines",
                name=f"{label} ({unit})",
                hovertemplate="%{x}<br>%{y:.2f} " + unit
            ))

            fig.update_layout(
                title=f"{station} | {label} | Agg: {agg} | Validity: {val}",
                xaxis_title="Time",
                yaxis_title=f"Concentration ({unit})",
                height=500,
                margin=dict(l=20, r=20, t=40, b=40),
                template="plotly_white"
            )

            fig.show()

            download_link = get_csv_download_link(df, label)
            display(download_link)

    plot_button.on_click(on_plot)

def get_csv_download_link(df, pollutant_label):
    # Rename value column to the pollutant name
    df_to_save = df.rename(columns={"value": pollutant_label, "timestamp": "timestamp"})

    # Convert to CSV and encode to base64
    buffer = io.StringIO()
    df_to_save.to_csv(buffer, index=False)
    b64 = base64.b64encode(buffer.getvalue().encode()).decode()

    # Create a clickable download link
    return widgets.HTML(
        f'<a download="air_quality_{pollutant_label}.csv" href="data:text/csv;base64,{b64}" target="_blank">📥 Download CSV for {pollutant_label}</a>'
    )

In [33]:
def on_file_upload(change):
    upload_output.clear_output()
    if not file_upload.value:
        with upload_output:
            print("Please upload a .h5 file.")
        return

    uploaded_file = file_upload.value[0]
    uploaded_filename = uploaded_file["name"]
    file_bytes = uploaded_file["content"]
    
    h5_path = "uploaded_voila_file.h5"
    with open(h5_path, "wb") as f:
        f.write(file_bytes)

    with upload_output:
        print(f"File uploaded: {uploaded_filename}")
    
    update_gui_from_file(h5_path)

    # if os.path.exists(h5_path):
    #     os.remove(h5_path)


In [34]:
file_upload.observe(on_file_upload, names="value")

ui = widgets.VBox([
    file_upload,
    upload_output,
    station_dropdown,
    pollutant_dropdown,
    plot_button,
    output_plot
])

display(ui)